# Load Prompt Registry Client
In the next example, following the previous Notebook on Prompt creation, we will create and upload a Prompt template to the Prompt Registry

In [ ]:
from gen_ai_hub.proxy import get_proxy_client
from gen_ai_hub.prompt_registry.client import PromptTemplateClient

from gen_ai_hub.prompt_registry.models.prompt_template import PromptTemplateSpec, PromptTemplate
from dotenv import load_dotenv
load_dotenv()

proxy_client = get_proxy_client(proxy_version="gen-ai-hub")
prompt_registry_client = PromptTemplateClient(proxy_client=proxy_client)
print("PromptTemplateClient initialized successfully.")

# Use camelCase for the prompt variable names
prompt_spec = PromptTemplateSpec(
    template=[
        PromptTemplate(role="system",
                       content="You are a helpful translation assistant."),
        PromptTemplate(role="user",
                       content="Translate the following text to {{?toLang}}: {{?text}}")
    ],
    defaults={"toLang": "German"}
)


resp = prompt_registry_client.create_prompt_template(
    scenario="translation",
    name="translate_text",
    version="0.1.0",
    prompt_template_spec=prompt_spec,
)
template_id = resp.id
print("Created template:", template_id)

PromptTemplateClient initialized successfully.
Created template: 9d419fab-c4c3-4443-a38f-d615e5ad39e6


# Define the LLM
The `LLM` class is used to configure and initialize a language model for generating text based on specific parameters. In this example, we'll use the gpt-4o

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

# Create Orchestration Configuration
The `OrchestrationConfig` class defines a configuration for integrating various modules, such as templates and language models, into a cohesive orchestration setup.

In [4]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.template_ref import TemplateRef

template_ref = TemplateRef.from_tuple(
    scenario="translation",
    name="translate_text",
    version="0.1.0",
)

config = OrchestrationConfig(
    template=template_ref,
    llm=llm,
)

# Run the Orchestration Request

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.template import TemplateValue

orchestration_service = OrchestrationService(config=config)

result = orchestration_service.run(template_values=[
    TemplateValue(name="toLang", value="French"),
    TemplateValue(name="text", value="The Orchestration Service is working!")
])

print(result.orchestration_result.choices[0].message.content)

Le service d'orchestration fonctionne !


In [ ]:
# Using default language value
result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])

print(result.orchestration_result.choices[0].message.content)

Der Orchestrierungsdienst funktioniert!
